**[Machine Learning Home Page](https://www.kaggle.com/learn/intro-to-machine-learning)**

---


# Introduction
Machine learning competitions are a great way to improve your data science skills and measure your progress. 

In this exercise, you will create and submit predictions for a Kaggle competition. You can then improve your model (e.g. by adding features) to improve and see how you stack up to others taking this micro-course.

The steps in this notebook are:
1. Build a Random Forest model with all of your data (**X** and **y**)
2. Read in the "test" data, which doesn't include values for the target.  Predict home values in the test data with your Random Forest model.
3. Submit those predictions to the competition and see your score.
4. Optionally, come back to see if you can improve your model by adding features or changing your model. Then you can resubmit to see how that stacks up on the competition leaderboard.

## Recap
Here's the code you've written so far. Start by running it again.

In [ ]:
# Code you have previously used to load data
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
from learntools.core import *



# Path of the file to read. We changed the directory structure to simplify submitting to a competition
iowa_file_path = '../input/train.csv'
home_data = pd.read_csv(iowa_file_path)
# Create target object and call it y
y = home_data.SalePrice
# Create X
features = ['LotArea', 'YearBuilt', '1stFlrSF', '2ndFlrSF', 'FullBath', 'BedroomAbvGr', 'TotRmsAbvGrd']
X = home_data[features]

# Split into validation and training data
train_X, val_X, train_y, val_y = train_test_split(X, y, random_state=1)

# Specify Model
iowa_model = DecisionTreeRegressor(random_state=1)
# Fit Model
iowa_model.fit(train_X, train_y)

# Make validation predictions and calculate mean absolute error
val_predictions = iowa_model.predict(val_X)
val_mae = mean_absolute_error(val_predictions, val_y)
print("Validation MAE when not specifying max_leaf_nodes: {:,.0f}".format(val_mae))

# Using best value for max_leaf_nodes
iowa_model = DecisionTreeRegressor(max_leaf_nodes=100, random_state=1)
iowa_model.fit(train_X, train_y)
val_predictions = iowa_model.predict(val_X)
val_mae = mean_absolute_error(val_predictions, val_y)
print("Validation MAE for best value of max_leaf_nodes: {:,.0f}".format(val_mae))

# Define the model. Set random_state to 1
rf_model = RandomForestRegressor(random_state=1)
rf_model.fit(train_X, train_y)
rf_val_predictions = rf_model.predict(val_X)
rf_val_mae = mean_absolute_error(rf_val_predictions, val_y)

print("Validation MAE for Random Forest Model: {:,.0f}".format(rf_val_mae))


# Creating a Model For the Competition

Build a Random Forest model and train it on all of **X** and **y**.  

**Ok, I'm going to train the model with all X and y**

In [ ]:
rf_model_all = RandomForestRegressor(random_state=1)

rf_model_all.fit(X, y)

In [ ]:
home_data = pd.read_csv(iowa_file_path, index_col=["Id"])

In [ ]:
home_data.dtypes.value_counts()

In [ ]:
home_data.head()

In [ ]:
lista = list(home_data.select_dtypes(include=[pd.np.object]).columns)
lista.append("OverallQual")
lista.append("OverallCond")
lista.append("MSSubClass")
home_data.loc[:,lista] = home_data.loc[:,lista].astype("category")   

In [ ]:
pd.get_dummies(home_data.select_dtypes("category")).head()

In [ ]:
home_data.select_dtypes(["float64","int64"]).head()

In [ ]:
new_df = pd.concat([home_data.select_dtypes(["float64","int64"]),pd.get_dummies(home_data.select_dtypes("category"))], axis = 1)

In [ ]:
new_df = new_df.dropna()

In [ ]:
train_X, val_X, train_y, val_y = train_test_split(new_df.drop(['SalePrice'], axis=1), new_df.loc[:,'SalePrice'], random_state=1)

In [ ]:
# To improve accuracy, create a new Random Forest model which you will train on all training data
rf_model_on_full_data = RandomForestRegressor(random_state=1)

# fit rf_model_on_full_data on all data from the training data
rf_model_on_full_data.fit(train_X, train_y)

In [ ]:
val_predictions = rf_model_on_full_data.predict(val_X)
val_mae = mean_absolute_error(val_predictions, val_y)
print("Validation MAE for all data: {:,.0f}".format(val_mae))

# Make Predictions
Read the file of "test" data. And apply your model to make predictions

In [ ]:
# path to file you will use for predictions
test_data_path = '../input/test.csv'

# read test data file using pandas
test_data = pd.read_csv(test_data_path, index_col=["Id"])

**In the next code line I'm going to test using the model with all X features**

In [ ]:
features = ['LotArea', 'YearBuilt', '1stFlrSF', '2ndFlrSF', 'FullBath', 'BedroomAbvGr', 'TotRmsAbvGrd']

test_preds_all = rf_model_all.predict(test_data.loc[:,features])

# The lines below shows how to save predictions in format used for competition scoring
# Just uncomment them.

output = pd.DataFrame({'Id': test_data.index,
                       'SalePrice': test_preds_all})
output.to_csv('submission_all.csv', index=False)

In the next chunks are proposed another way to train the model

In [ ]:
lista = list(test_data.select_dtypes(include=[pd.np.object]).columns)
lista.append("OverallQual")
lista.append("OverallCond")
lista.append("MSSubClass")
test_data.loc[:,lista] = test_data.loc[:,lista].astype("category")   

In [ ]:
test_data.select_dtypes(["int64","float64"]).isnull().sum().sort_values(ascending=False)

In [ ]:
test_data.loc[:,list(test_data.select_dtypes(["int64","float64"]).columns)] = test_data.loc[:,list(test_data.select_dtypes(["int64","float64"]).columns)].fillna(test_data.loc[:,list(test_data.select_dtypes(["int64","float64"]).columns)].mean())

In [ ]:
test_data.select_dtypes(["int64","float64"]).isnull().sum().sort_values(ascending=False)

In [ ]:
pd.get_dummies(test_data.select_dtypes("category")).head()

In [ ]:
final_train, final_test = pd.get_dummies(home_data.select_dtypes("category")).align(pd.get_dummies(test_data.select_dtypes("category")),
                                                                    join='left', 
                                                                    axis=1)

In [ ]:
final_train.shape

In [ ]:
new_testdf = pd.concat([test_data.select_dtypes(["float64","int64"]),final_test], axis = 1)

In [ ]:
new_testdf.columns

In [ ]:
new_testdf.isnull().sum().sort_values(ascending=False)

In [ ]:
new_testdf = new_testdf.fillna(0)

In [ ]:
new_testdf.isnull().sum().sum()

In [ ]:

# create test_X which comes from test_data but includes only the columns you used for prediction.
# The list of columns is stored in a variable called features
test_X = new_testdf.copy()

# make predictions which we will submit. 
test_preds = rf_model_on_full_data.predict(test_X)

# The lines below shows how to save predictions in format used for competition scoring
# Just uncomment them.

output = pd.DataFrame({'Id': test_data.index,
                       'SalePrice': test_preds})
output.to_csv('submission.csv', index=False)

**Prediction for the model with all X**

# Test Your Work
After filling in the code above:
1. Click the **Commit** button. 
2. After your code has finished running, click the "Open Version" button.  This brings you into the "viewer mode" for your notebook. You will need to scroll down to get back to these instructions.
3. Click **Output** button on the left of your screen. 

This will bring you to a part of the screen that looks like this: 
![](https://imgur.com/a/QRHL7Uv)

Select the button to submit and you will see your score. You have now successfully submitted to the competition.

4. If you want to keep working to improve your model, select the edit button. Then you can change your model and repeat the process to submit again. There's a lot of room to improve your model, and you will climb up the leaderboard as you work.

# Continuing Your Progress
There are many ways to improve your model, and **experimenting is a great way to learn at this point.**

The best way to improve your model is to add features.  Look at the list of columns and think about what might affect home prices.  Some features will cause errors because of issues like missing values or non-numeric data types. 

The [Intermediate Machine Learning](https://www.kaggle.com/learn/intermediate-machine-learning) micro-course will teach you how to handle these types of features. You will also learn to use **xgboost**, a technique giving even better accuracy than Random Forest.


# Other Micro-Courses
The **[Pandas Micro-Course](https://kaggle.com/Learn/Pandas)** will give you the data manipulation skills to quickly go from conceptual idea to implementation in your data science projects. 

You are also ready for the **[Deep Learning](https://kaggle.com/Learn/Deep-Learning)** micro-course, where you will build models with better-than-human level performance at computer vision tasks.

---
**[Machine Learning Home Page](https://www.kaggle.com/learn/intro-to-machine-learning)**





*Have questions or comments? Visit the [Learn Discussion forum](https://www.kaggle.com/learn-forum) to chat with other Learners.*